In [13]:
from sklearn.preprocessing import StandardScaler, MinMaxScaler
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt

import sklearn
from sklearn.manifold import TSNE
from sklearn.decomposition import PCA
from umap import UMAP
from sklearn import datasets
import kmapper as km

In [14]:
data = pd.read_csv('./data/Test_Data_1102_enumerated_filledin.csv')
y = data['CaseCtrl']
X = data.values

In [15]:
print(X.shape)

(25489, 148)


In [17]:
# Initialize to use t-SNE
mapper = km.KeplerMapper(verbose=0)

#projected_data = mapper.fit_transform(X, projection=TSNE(perplexity=50))

lens = mapper.fit_transform(X[:, 0])

In [18]:
# Create the graph using DBSCAN as the clustering algorithm
graph = mapper.map(
    lens,
    X,
    clusterer=sklearn.cluster.DBSCAN(eps=0.2, min_samples=20),
    cover=km.Cover(n_cubes=20, perc_overlap=0.30),
)

In [19]:
mapper.visualize(
    graph,
    title="Mapper visualization",
    path_html="mapper.html",
    custom_tooltips=y,
)

Exception: Visualize requires a mapper with more than 0 nodes. 
It is possible that the constructed mapper could have been constructed with bad parameters. This can occasionally happens when using the default clustering algorithm. Try changing `eps` or `min_samples` in the DBSCAN clustering algorithm.